In [1]:
import requests
import pandas as pd
import numpy as np
#import eurostat
import json
import os
import datetime as dt
import openpyxl

In [2]:
# ==== CONFIG ====
BASE_DIR = '/Users/lorenz/Projects/GaR/'
DATA_DIR = os.path.join(BASE_DIR, 'src')
os.makedirs(DATA_DIR, exist_ok=True)

# INPUT FILES (update if needed)
IP_FILE = os.path.join(DATA_DIR,'estat_sts_inpr_m_filtered_en-3.csv')
GAS_FILE = os.path.join(DATA_DIR, 'gas_data.csv')
OIL_FILE = os.path.join(DATA_DIR, 'oilSupplyNewsShocks_2024M06.xlsx')
BRENT_FILE = os.path.join(DATA_DIR, 'brenteu.xlsx')

In [3]:
# ==== LOAD INDUSTRIAL PRODUCTION ====
ip = pd.read_csv(IP_FILE)
ip['Time'] = pd.to_datetime(ip['TIME_PERIOD'], format="%Y-%m", errors='coerce')
ip = ip.dropna(subset=['Time'])
ip = ip.set_index('Time')
ip = ip[['nace_r2', 'geo', 'OBS_VALUE', 'unit']]

In [4]:
# ==== GAS PRICES ====
gasp = pd.read_csv(GAS_FILE)
gasp['Prices'] = gasp['LSG Natural Gas TTF NL 1st Fut. Day - SETT. PRICE']
gasp['Time'] = pd.to_datetime(gasp['Time'], format="%m/%d/%y")
gas_month = gasp.groupby(gasp['Time'].dt.to_period('M')).mean(numeric_only=True)
gas_month.index = pd.to_datetime(gas_month.index.to_timestamp())
gas_month['Prices_t1'] = gas_month['Prices'].shift(1)
gas_month['Prices_t1'] = gas_month['Prices_t1'].interpolate(method="nearest").ffill().bfill()


In [5]:
# ==== OIL SHOCKS ====
oil_df = pd.read_excel(OIL_FILE, sheet_name="Monthly")
oil_df['Date'] = pd.to_datetime(oil_df['Date'].str.replace("M", ""), format="%Y%m")
oil_df = oil_df[(oil_df['Date'] >= "2004-01-01") & (oil_df['Date'] <= "2023-12-31")]
oil_df = oil_df.rename(columns={"Date": "Time"})
oil_df = oil_df.groupby(oil_df['Time'].dt.to_period('M')).mean(numeric_only=True)
oil_df.index = pd.to_datetime(oil_df.index.to_timestamp())
oil_fixed = oil_df.rename(columns={
    "Oil supply surprise series": "OilSurprise",
    "Oil supply news shock": "OilNewsShock"
})[["OilNewsShock", "OilSurprise"]]

In [6]:
# ==== BRENT ====
brent_df = pd.read_excel(BRENT_FILE, sheet_name="Foglio1")
brent_df['Date'] = pd.to_datetime(brent_df['Date'])
brent_df = brent_df.rename(columns={'Europe Brent Spot FOB U$/BBL Daily': 'BRENT_PRICE'})
brent_month = brent_df.groupby(brent_df['Date'].dt.to_period('M')).mean(numeric_only=True)
brent_month.index = pd.to_datetime(brent_month.index.to_timestamp())
brent_month['BRENT_PRICE_t1'] = brent_month['BRENT_PRICE'].shift(1)
brent_month['BRENT_PRICE_t1'] = brent_month['BRENT_PRICE_t1'].interpolate(method="nearest").ffill().bfill()

# Align all indices to month start for consistent joins
ip.index = pd.to_datetime(ip.index.to_period('M').astype(str))
gas_month.index = pd.to_datetime(gas_month.index.to_period('M').astype(str))
oil_fixed.index = pd.to_datetime(oil_fixed.index.to_period('M').astype(str))
brent_month.index = pd.to_datetime(brent_month.index.to_period('M').astype(str))

# ==== MERGE BRENT AND OIL ====
brent_only = brent_month[["BRENT_PRICE"]].copy()
brent_only = brent_only.join(oil_fixed, how='left')
brent_only.index.name = "Time"
brent_only = brent_only.reset_index()
brent_only['Time'] = pd.to_datetime(brent_only['Time'])
brent_only.to_csv(os.path.join(BASE_DIR, "oil_only.csv"), index=False)


In [7]:
# Align all indices to month start for consistent joins
ip.index = pd.to_datetime(ip.index.to_period('M').astype(str))
gas_month.index = pd.to_datetime(gas_month.index.to_period('M').astype(str))
oil_fixed.index = pd.to_datetime(oil_fixed.index.to_period('M').astype(str))
brent_month.index = pd.to_datetime(brent_month.index.to_period('M').astype(str))


In [8]:
# ==== LOOP THROUGH COMBINATIONS ====
master = []
unit_filter = 'Index, 2021=100'

for geo in ip['geo'].unique():
    for sector in ip['nace_r2'].unique():
        subset = ip[(ip['geo'] == geo) & (ip['nace_r2'] == sector) & (ip['unit'] == unit_filter)]
        if subset.empty:
            continue

        subset = subset.loc['2004-01-01':'2023-12-31'].copy()
        if subset.empty:
            continue

        subset = subset.rename(columns={'OBS_VALUE': 'IP'})
        subset = subset.sort_index()

        subset['IP_t1'] = subset['IP'].shift(1)
        subset['IP_t1'] = subset['IP_t1'].interpolate(method="nearest").ffill().bfill()


        subset = subset.join(gas_month[['Prices', 'Prices_t1']], how='left')
        subset = subset.rename(columns={
            'Prices': 'GAS_PRICE',
            'Prices_t1': 'GAS_PRICE_t1'
        })
        subset = subset.join(oil_fixed, how='left')
        subset = subset.join(brent_month[['BRENT_PRICE', 'BRENT_PRICE_t1']], how='left')

        safe_geo = geo.replace(" ", "_").replace("/", "_")
        safe_sector = sector.replace(" ", "_").replace("/", "_")
        out_file = os.path.join(DATA_DIR, f"{safe_geo}_{safe_sector}_ip_merged.csv")
        subset.to_csv(out_file, index=True)

        master.append(subset)



# Combine all into master dataset
if master:
    df_master = pd.concat(master)
    df_master.to_csv(os.path.join(DATA_DIR, "df_final_all.csv"))
    print("Master merged dataset saved.")
else:
    print("No data found for the specified filters.")

Master merged dataset saved.
